In [ ]:
pip install beautifulsoup4 pandas html5lib

In [ ]:
pip install --upgrade html5lib lxml


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re

class AuditReportParser:
    def parse_html(self, file_path):
        """
        HTML 파일을 읽고 BeautifulSoup 객체로 파싱하여 반환합니다.
        
        Args:
            file_path (str): HTML 파일 경로.
            
        Returns:
            BeautifulSoup: 파싱된 HTML 객체.
            
        Raises:
            IOError: 파일을 읽을 수 없을 때 발생.
        """
        try:
            # 감사보고서 파일의 인코딩이 'euc-kr'인 경우가 많습니다.
            with open(file_path, 'r', encoding='euc-kr') as f:
                html_content = f.read()
            return BeautifulSoup(html_content, 'html.parser')
        except IOError as e:
            print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
            return None
    
    def extract_sections(self, parsed_html):
        """
        HTML에서 <p> 태그의 텍스트를 문단별로 추출하여 리스트로 반환합니다.
        
        Args:
            parsed_html (BeautifulSoup): parse_html() 메서드가 반환한 객체.
            
        Returns:
            list: 각 <p> 태그의 텍스트가 담긴 문자열 리스트.
        """
        if not parsed_html:
            return []
            
        paragraphs = parsed_html.find_all('p')
        parsed_texts = [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)]
        
        return parsed_texts
    
    def extract_tables(self, parsed_html):
        """
        HTML에서 class="TABLE" 속성을 가진 테이블을 DataFrame 리스트로 추출합니다.
        
        Args:
            parsed_html (BeautifulSoup): parse_html() 메서드가 반환한 객체.
            
        Returns:
            list: 각 테이블을 변환한 pandas DataFrame 객체 리스트.
        """
        if not parsed_html:
            return []
            
        tables = parsed_html.find_all('table', class_='TABLE')
        dfs = []
        
        for table in tables:
            rows = table.find_all('tr')
            
            if not rows:
                continue
                
            data = []
            max_cols = 0
            for row in rows:
                cols = row.find_all(['td', 'th'])
                row_data = [col.get_text(strip=True) for col in cols]
                data.append(row_data)
                if len(row_data) > max_cols:
                    max_cols = len(row_data)
            
            padded_data = [row + [''] * (max_cols - len(row)) for row in data]
            
            if padded_data and len(padded_data) > 1:
                df = pd.DataFrame(padded_data[1:], columns=padded_data[0])
                dfs.append(df)
            elif padded_data:
                df = pd.DataFrame(padded_data)
                dfs.append(df)
        
        return dfs

    def normalize_text(self, text):
        """
        금융 텍스트를 정규화하고 정제합니다.
        
        Args:
            text (str): 정제할 텍스트.
            
        Returns:
            str: 정규화된 텍스트.
        """
        # 불필요한 공백 문자(줄바꿈, 탭 등)를 단일 공백으로 치환
        text = re.sub(r'\s+', ' ', text)
        # 괄호와 그 안의 내용을 제거 (예: (주)삼성전자)
        text = re.sub(r'\([^)]*\)', '', text)
        # 특수문자 제거 (한글, 영어, 숫자, 공백 제외)
        text = re.sub(r'[^가-힣a-zA-Z0-9\s.,]', '', text)
        # 쉼표 뒤에 공백 추가
        text = re.sub(r',', ', ', text)
        # 여러 개의 공백을 단일 공백으로
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text

    def extract_inventory_table(self, parsed_html):
        """
        HTML에서 '재고자산 내역' 테이블을 찾아 DataFrame으로 반환합니다.
        """
        search_texts = [
            '보고기간종료일 현재 재고자산의 내역은 다음과 같습니다.',
            '재고자산의 내역은 다음과 같습니다.',
        ]

        for text in search_texts:
            text_elements = parsed_html.find_all(lambda tag: tag.name in ['p', 'span'] and text in tag.get_text(strip=True))

            for element in text_elements:
                next_table = element.find_next('table', class_='TABLE')

                if next_table:
                    # extract_tables 메서드의 로직을 재사용하여 DataFrame으로 변환
                    rows = next_table.find_all('tr')
                    data = []
                    max_cols = 0
                    for row in rows:
                        cols = row.find_all(['td', 'th'])
                        row_data = [col.get_text(strip=True) for col in cols]
                        data.append(row_data)
                        if len(row_data) > max_cols:
                            max_cols = len(row_data)

                    padded_data = [row + [''] * (max_cols - len(row)) for row in data]

                    if padded_data and len(padded_data) >= 3:
                        # 임시 DataFrame 생성 (헤더 없이 데이터만 가져옴)
                        df = pd.DataFrame(padded_data[2:])

                        # 하드코딩된 상위 헤더
                        full_upper_header = ['구분', '당기말', '당기말', '당기말', '전기말', '전기말', '전기말']

                        # 하위 헤더를 HTML에서 가져옴
                        lower_header_raw = padded_data[1]

                        # 하위 헤더를 상위 헤더의 길이와 맞추기
                        # '구분' 헤더에 해당하는 빈 문자열을 추가
                        full_lower_header = [''] + lower_header_raw

                        # MultiIndex 생성 및 DataFrame에 적용
                        df.columns = pd.MultiIndex.from_tuples(list(zip(full_upper_header, full_lower_header)))

                        # '구분' 열을 인덱스로 설정
                        # MultiIndex에서 '구분' 열은 ('구분', '') 튜플로 접근해야 함
                        #df.set_index(('구분', ''), inplace=True)
                        df = df.drop('전기말', axis=1, level=0)
                        return df
                    elif padded_data:
                        # 헤더가 1개인 경우
                        return pd.DataFrame(padded_data[1:], columns=padded_data[0])

        return None

    def extract_investment_changes(self, parsed_html):
        """
        HTML에서 '종속기업, 관계기업 및 공동기업 투자의 변동내역' 테이블을 찾아 DataFrame으로 반환합니다.
        
        Args:
            parsed_html (BeautifulSoup): 파싱된 HTML 객체.
            
        Returns:
            pd.DataFrame: 변동내역 데이터가 담긴 DataFrame.
        """
        # 텍스트를 포함하는 p 또는 span 태그를 찾습니다.
        search_text = '가. 당기 및 전기 중 종속기업, 관계기업 및 공동기업 투자의 변동'
        
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and search_text in tag.get_text(strip=True)
        )

        if not title_element:
            print(f"'{search_text}' 텍스트를 찾을 수 없습니다.")
            return None
        
        # 텍스트 다음에 나오는 첫 번째 'TABLE' 클래스 테이블을 찾습니다.
        table_element = title_element.find_next('table', class_='TABLE')

        if not table_element:
            print("관련 테이블을 찾을 수 없습니다.")
            return None
        
        # 테이블 데이터 추출
        rows = table_element.find_all('tr')
        
        if len(rows) < 2:
            print("테이블에 충분한 데이터가 없습니다.")
            return None
            
        # 헤더와 데이터 추출
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]

        # DataFrame 생성 및 반환
        df = pd.DataFrame(data, columns=headers)
        if '전기' in df.columns:
            df = df.drop(columns=['전기'])
        
        return df
    def extract_major_investments(self, parsed_html):
        """
        HTML에서 '주요 관계기업 및 공동기업 투자 현황' 테이블을 찾아 DataFrame으로 반환합니다.
        
        Args:
            parsed_html (BeautifulSoup): 파싱된 HTML 객체.
            
        Returns:
            pd.DataFrame: 주요 투자 현황 데이터가 담긴 DataFrame.
        """
        search_texts = [
            '(1) 관계기업 투자',
            '투자 현황은 다음과 같습니다'
        ]
        
        for text in search_texts:
            title_element = parsed_html.find(
                lambda tag: tag.name in ['p', 'span'] and text in tag.get_text(strip=True)
            )
            
            if title_element:
                table_element = title_element.find_next('table', class_='TABLE')
                if table_element:
                    rows = table_element.find_all('tr')
                    if len(rows) < 2:
                        continue
                        
                    headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
                    data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
                    
                    df = pd.DataFrame(data, columns=headers)
                    
                    # 지분율 컬럼 전처리
                    if '지분율(%)' in df.columns:
                        df['지분율(%)'] = pd.to_numeric(df['지분율(%)'], errors='coerce')
                    
                    return df
        
        print("주요 관계기업 투자 현황 테이블을 찾을 수 없습니다.")
        return None

    def extract_subsidiaries(self, parsed_html):
        """
        HTML에서 '(1) 주요 종속기업' 테이블을 찾아 두 가지 유형의 헤더를 모두 처리하여 DataFrame으로 반환합니다.
        """
        search_text = '(1) 주요 종속기업'
        
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and search_text in tag.get_text(strip=True)
        )

        if not title_element:
            return None
        
        table_element = title_element.find_next('table', class_='TABLE')
        
        if not table_element:
            return None
        
        rows = table_element.find_all('tr')
        
        if len(rows) < 2:
            return None
            
        # 첫 번째 행의 'rowspan' 속성으로 다중 헤더 여부 판단
        first_row_ths = rows[0].find_all(['th', 'td'])
        is_multi_header = any(th.get('rowspan') for th in first_row_ths)

        # 헤더와 데이터 추출
        if is_multi_header and len(rows) >= 2:
            # 다중 헤더 처리: 두 번째 행의 헤더를 사용
            # 첫 번째 행은 버리고 두 번째 행을 헤더로 사용
            lower_header_elements = rows[1].find_all(['th', 'td'])
            headers = [th.get_text(strip=True) for th in lower_header_elements]
            
            # 첫 번째 열에 해당하는 헤더가 없으면 '기업명'으로 채움
            if len(headers) != len(rows[2].find_all('td')):
                headers = ['기업명'] + [h for h in headers]
                
            data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[2:]]
            
            df = pd.DataFrame(data, columns=headers)
            
        else:
            # 단일 헤더 처리
            headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
            data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
            
            df = pd.DataFrame(data, columns=headers)
            
        # 공통 전처리: 불필요한 텍스트 및 기호 제거
        df.columns = df.columns.astype(str).str.replace(r'\(.*?\)', '', regex=True).str.strip()
        df.rename(columns={'기업명': '기업명', '당기순이익(손실)': '당기순이익'}, inplace=True)
        
        # 숫자형 컬럼 전처리 (콤마와 하이픈 제거 후 float 변환)
        for col in df.columns[1:]:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '').str.replace('-', '0'), errors='coerce')

        return df
    def _extract_table_by_text(self, parsed_html, search_text):
        """
        특정 텍스트 다음에 오는 단일 헤더 테이블을 파싱합니다.
        """
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and search_text in tag.get_text(strip=True)
        )
        if not title_element:
            return None
        
        table_element = title_element.find_next('table', class_='TABLE')
        if not table_element:
            return None
            
        rows = table_element.find_all('tr')
        if len(rows) < 2:
            return None
        
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
        
        return pd.DataFrame(data, columns=headers)
    
    def _extract_single_financial_table(self, parsed_html, search_text):
        """
        특정 텍스트 다음에 오는 테이블을 파싱합니다.
        """
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span', 'td'] and search_text in tag.get_text(strip=True)
        )
        if not title_element:
            return None
        
        table_element = title_element.find_next('table', class_='TABLE')
        if not table_element:
            table_element = title_element.find_parent('table', class_='TABLE')
            if not table_element:
                return None
        
        rows = table_element.find_all('tr')
        if len(rows) < 2:
            return None
        
        header_row_index = -1
        for i, row in enumerate(rows):
            if any('구분' in th.get_text(strip=True) for th in row.find_all(['th', 'td'])):
                header_row_index = i
                break

        if header_row_index == -1:
            return None

        headers = [th.get_text(strip=True) for th in rows[header_row_index].find_all(['th', 'td'])]
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[header_row_index+1:]]
        
        df = pd.DataFrame(data, columns=headers)
        
        df = df[~df['구분'].astype(str).str.contains('요약', na=False)].copy()
        
        return df
    
    def _extract_sub_table(self, parsed_html, table_title):
        """
        특정 제목을 가진 하위 테이블을 파싱하는 도우미 함수.
        """
        title_element = parsed_html.find(
            lambda tag: tag.name in ['td'] and table_title in tag.get_text(strip=True)
        )
        if not title_element:
            return None
            
        table_element = title_element.find_parent('table', class_='TABLE')
        if not table_element:
            return None

        rows = table_element.find_all('tr')
        if not rows or len(rows) < 2:
            return None
        
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
        df = pd.DataFrame(data, columns=headers)
        
        return df   

    def _parse_table_content(self, table_element):
        """BeautifulSoup 테이블 객체를 DataFrame으로 파싱하는 도우미 함수."""
        if not table_element:
            return None
        
        rows = table_element.find_all('tr')
        if not rows or len(rows) < 2:
            return None
        
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
        
        return pd.DataFrame(data, columns=headers)
    
    def _parse_sub_table_by_title(self, parsed_html, table_title):
        """특정 제목을 가진 하위 테이블을 파싱하는 도우미 함수."""
        title_element = parsed_html.find(
            lambda tag: tag.name in ['td'] and table_title in tag.get_text(strip=True)
        )
        if not title_element:
            return None
            
        table_element = title_element.find_parent('table', class_='TABLE')
        if not table_element:
            return None

        rows = table_element.find_all('tr')
        if not rows or len(rows) < 2:
            return None
        
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
        
        return pd.DataFrame(data, columns=headers)
    def _parse_table_content(self, table_element):
        """BeautifulSoup 테이블 객체를 DataFrame으로 파싱하는 도우미 함수."""
        if not table_element:
            return None
        rows = table_element.find_all('tr')
        if not rows or len(rows) < 2:
            return None
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
        return pd.DataFrame(data, columns=headers)

        
    def extract_financial_info_double(self, parsed_html):
        """관계기업 재무정보 테이블을 파싱하여 DataFrame으로 반환합니다."""
        search_pattern_main = re.compile(r'\(2\)\s*주요\s*관계기업')
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and re.search(search_pattern_main, tag.get_text(strip=True))
        )
        if not title_element:
            print("경고: '주요 관계기업' 제목을 찾을 수 없습니다.")
            return None
            
        # 두 개의 테이블이 연속해서 있는지 확인하는 로직
        tables_after_title = title_element.find_all_next('table', class_='TABLE', limit=2)
        is_two_tables_case = len(tables_after_title) >= 2

        if is_two_tables_case:
            table_element_1 = tables_after_title[0]
            table_element_2 = tables_after_title[1]

            if not table_element_1 or not table_element_2:
                print("경고: 두 개의 분리된 테이블을 찾을 수 없습니다.")
                return None
            
            df_bs = self._parse_table_content(table_element_1)
            df_is = self._parse_table_content(table_element_2)

            if df_bs is None or df_is is None:
                return None
            
            samsung_card_col_name = next((col for col in df_bs.columns if '삼성카드' in col), None)
            if samsung_card_col_name and '유동자산' in df_bs['구분'].values and '비유동자산' in df_bs['구분'].values:
                print("삼성카드 데이터 보정 중...")
                df_bs.loc[df_bs['구분'] == '비유동자산', samsung_card_col_name] = df_bs.loc[df_bs['구분'] == '유동자산', samsung_card_col_name].values[0]
                df_bs.loc[df_bs['구분'] == '비유동부채', samsung_card_col_name] = df_bs.loc[df_bs['구분'] == '유동부채', samsung_card_col_name].values[0]
            
            df_bs.set_index('구분', inplace=True)
            df_is.set_index('구분', inplace=True)
            final_df = pd.concat([df_bs, df_is], axis=0, join='outer')
            final_df.index.name = None
            final_df = final_df.T
            
            final_df.loc[:, '순자산'] = pd.NA
            final_df.index = final_df.index.str.replace(r'[\s\(\)\*]', '', regex=True)
            
            #for col in final_df.columns:
                #final_df[col] = pd.to_numeric(final_df[col].astype(str).str.replace(',', '', regex=False).str.replace('-', '0', regex=False).str.replace('(', '-', regex=False).str.replace(')', '', regex=False), errors='coerce')
            
            return final_df

        
    def extract_financial_info_single(self, parsed_html):
        """
        통합된 '관계기업의 재무정보' 테이블을 파싱하여 DataFrame으로 반환합니다.
        """
        search_pattern = re.compile(r'\(2\)\s*주요\s*관계기업')
        
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and re.search(search_pattern, tag.get_text(strip=True))
        )
        if not title_element:
            print("경고: '주요 관계기업' 제목을 찾을 수 없습니다.")
            return None
        
        table_element = title_element.find_next('table', class_='TABLE')
        if not table_element:
            print("경고: 관련 테이블을 찾을 수 없습니다.")
            return None
            
        rows = table_element.find_all('tr') 
        if len(rows) < 2:
            print("경고: 테이블에 충분한 행이 없습니다.")
            return None

        # 헤더 행의 개수를 기준으로 파싱 로직 분기
        if len(rows) >= 3 and len(rows[1].find_all(['th', 'td'])) > 1:
            # 헤더 행이 2개인 경우
            company_names_row = rows[1]
            data_start_row = 2
        else:
            # 헤더 행이 1개인 경우
            company_names_row = rows[0]
            data_start_row = 1
        
        # 1. 기업명(컬럼) 추출: <th>와 <td> 태그를 모두 찾아 '구분'을 제외
        company_names = []
        for elem in company_names_row.find_all(['th', 'td']):
            elem_text = elem.get_text(strip=True)
            if '구분' not in elem_text:
                company_names.append(elem_text)
        
        # 2. 데이터 및 재무 항목(인덱스) 추출
        financial_items = []
        financial_data = []

        clean_pattern = re.compile(r'[\s\(\)\*]')

        for row in rows[data_start_row:]:
            tds = row.find_all('td')
            if '요약' in tds[0].get_text(strip=True):
                continue
            
            item_name = re.sub(clean_pattern, '', tds[0].get_text(strip=True))
            item_data = [td.get_text(strip=True) for td in tds[1:]]
            
            if len(item_data) != len(company_names):
                print(f"경고: 데이터 행의 열 개수({len(item_data)})와 기업명 개수({len(company_names)})가 불일치합니다.")
                continue

            financial_items.append(item_name)
            financial_data.append(item_data)
        
        if not financial_data:
            print("경고: 유효한 재무 데이터 행을 찾을 수 없습니다. DataFrame 생성 실패.")
            return None
        
        # 3. DataFrame 생성
        df = pd.DataFrame(financial_data, columns=company_names, index=financial_items)

        # 4. '순자산' 열 추가 (요청에 따라)
        df.loc['순자산', :] = pd.NA
        
        # 5. 숫자형 데이터 전처리
        for col in df.columns:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '').str.replace('-', '0').str.replace('(', '-').str.replace(')', ''), errors='coerce')
        
        # 최종적으로 행과 열을 뒤집어 원하는 형태로 만듭니다.
        df = df.T
        
        return df
    def extract_financial_info(self, parsed_html, year):
        """
        관계기업 재무정보 테이블의 구조를 파악하고 적절한 파싱 메서드를 호출합니다.
        """
        if year >= 2019:
            print(f"{year}년: 통합 테이블로 간주하고 파싱을 진행합니다.")
            return self.extract_financial_info_single(parsed_html)
        else:
            print(f"{year}년: 분리된 두 개의 테이블로 간주하고 파싱을 진행합니다.")
            return self.extract_financial_info_double(parsed_html)
    def extract_specific_investment_table(self,parsed_html):
        """
        첫 번째 헤더 행을 무시하고, 두 번째 행을 헤더로 사용하여
        복합 헤더 테이블을 DataFrame으로 파싱합니다.
        
        Args:
            parsed_html (BeautifulSoup): 파싱된 HTML 객체.
            
        Returns:
            pd.DataFrame: 파싱된 데이터가 담긴 DataFrame.
        """
        search_text = "관계기업 투자주식의 내역은 다음과 같습니다."
    
        # 텍스트를 포함하는 태그를 찾습니다.
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and search_text in tag.get_text(strip=True)
        )
        
        if not title_element:
            print(f"'{search_text}' 텍스트를 찾을 수 없습니다.")
            return None
        
        table = title_element.find_next('table', class_='TABLE')
        
        if not table:
            print("class='TABLE'인 테이블을 찾을 수 없습니다.")
            return None

        # 모든 행을 추출합니다.
        rows = table.find_all('tr')
        
        if len(rows) < 2:
            print("테이블에 충분한 데이터가 없습니다.")
            return None
        
        # 두 번째 행을 헤더로 사용합니다.
        # 첫 번째 행은 무시합니다.
        headers = [th.get_text(strip=True) for th in rows[1].find_all(['th', 'td'])]
        
        # 세 번째 행부터 데이터로 사용합니다.
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[2:]]
        
        # '구분'에 해당하는 첫 번째 헤더가 누락되었을 수 있으므로 추가합니다.
        # 제공된 HTML 구조에서는 두 번째 행에 '구분' 헤더가 없으므로 수동으로 추가합니다.
        headers = ['구분'] + headers
        
        # 데이터 행도 첫 번째 열을 추가해야 함 (만약 테이블 구조에 따라 필요하다면)
        # 하지만 제공된 HTML에서는 데이터가 6개 열이므로 그대로 사용합니다.
        
        # 데이터프레임 생성
        df = pd.DataFrame(data, columns=headers)
        df = df.iloc[:, :-2]
        return df
    def extract_tangible_assets_table(self,parsed_html):
        # 띄어쓰기를 허용하는 정규표현식 패턴
        search_pattern = re.compile(r'유형자산의\s*변동\s*내역은\s*다음과\s*같습니다\.')

        # 정규표현식 패턴을 사용하여 텍스트가 포함된 태그를 찾습니다.
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and search_pattern.search(tag.get_text(strip=True))
        )
        
        if not title_element:
            print(f"'{search_pattern}' 텍스트를 찾을 수 없습니다.")
            return None
        
        table_element = title_element.find_next('table', class_='TABLE')
        
        if not table_element:
            print("class='TABLE'인 테이블을 찾을 수 없습니다.")
            return None

        # 모든 행을 추출합니다.
        rows = table_element.find_all('tr')
        
        if len(rows) < 2:
            print("테이블에 충분한 데이터가 없습니다.")
            return None
        
        # 두 번째 행을 헤더로 사용합니다.
        # 첫 번째 행은 무시합니다.
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
        
        # 세 번째 행부터 데이터로 사용합니다.
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
        
        # '구분'에 해당하는 첫 번째 헤더가 누락되었을 수 있으므로 추가합니다.
        # 제공된 HTML 구조에서는 두 번째 행에 '구분' 헤더가 없으므로 수동으로 추가합니다.
 #       headers = ['구분'] + headers
        
        # 데이터 행도 첫 번째 열을 추가해야 함 (만약 테이블 구조에 따라 필요하다면)
        # 하지만 제공된 HTML에서는 데이터가 6개 열이므로 그대로 사용합니다.
        
        # 데이터프레임 생성
        df = pd.DataFrame(data, columns=headers)
        return df
    def extract_intangible_assets_table(self,parsed_html):
        # 띄어쓰기를 허용하는 정규표현식 패턴
        search_pattern = re.compile(r'무형자산의\s*변동\s*내역은\s*다음과\s*같습니다\.')

        # 정규표현식 패턴을 사용하여 텍스트가 포함된 태그를 찾습니다.
        title_element = parsed_html.find(
            lambda tag: tag.name in ['p', 'span'] and search_pattern.search(tag.get_text(strip=True))
        )
        
        if not title_element:
            print(f"'{search_pattern}' 텍스트를 찾을 수 없습니다.")
            return None
        
        table_element = title_element.find_next('table', class_='TABLE')
        
        if not table_element:
            print("class='TABLE'인 테이블을 찾을 수 없습니다.")
            return None

        # 모든 행을 추출합니다.
        rows = table_element.find_all('tr')
        
        if len(rows) < 2:
            print("테이블에 충분한 데이터가 없습니다.")
            return None
        
        # 두 번째 행을 헤더로 사용합니다.
        # 첫 번째 행은 무시합니다.
        headers = [th.get_text(strip=True) for th in rows[0].find_all(['th', 'td'])]
        
        # 세 번째 행부터 데이터로 사용합니다.
        data = [[td.get_text(strip=True) for td in row.find_all('td')] for row in rows[1:]]
        
        # '구분'에 해당하는 첫 번째 헤더가 누락되었을 수 있으므로 추가합니다.
        # 제공된 HTML 구조에서는 두 번째 행에 '구분' 헤더가 없으므로 수동으로 추가합니다.
 #       headers = ['구분'] + headers
        
        # 데이터 행도 첫 번째 열을 추가해야 함 (만약 테이블 구조에 따라 필요하다면)
        # 하지만 제공된 HTML에서는 데이터가 6개 열이므로 그대로 사용합니다.
        
        # 데이터프레임 생성
        df = pd.DataFrame(data, columns=headers)
        return df

In [ ]:

# 클래스 인스턴스 생성
parser = AuditReportParser()

# 1. HTML 파일 파싱
file_path = '삼성전자_감사보고서_2014_2024/감사보고서_2014.htm'
parsed_html = parser.parse_html(file_path)

if parsed_html:
    # 2. 섹션별 텍스트 추출
    sections = parser.extract_sections(parsed_html)
    print("--- 추출된 문단 정보 (일부) ---")
    for i, section in enumerate(sections[:5]): # 상위 5개 문단만 출력
        normalized_section = parser.normalize_text(section)
        print(f"[{i+1}] {normalized_section}")
    print("-" * 20)
    
    # 3. 테이블 데이터 추출
    tables = parser.extract_tables(parsed_html)
    print("--- 추출된 테이블 정보 (첫 번째 테이블) ---")
    if tables:
        print(tables[0].head())
    else:
        print("파싱된 테이블이 없습니다.")

In [ ]:
if parsed_html:
    # 2. 섹션별 텍스트 추출 및 정규화
    sections = parser.extract_sections(parsed_html)
    print(f"총 {len(sections)}개의 문단이 파싱되었습니다.")
    
    print("\n--- 파싱된 문단 (일부) ---")
    for i, section in enumerate(sections[:10]):
        normalized_section = parser.normalize_text(section)
        print(f"[{i+1}] {normalized_section}")

In [ ]:
import os
def browse_tables(file_path):
    parser = AuditReportParser()
    parsed_html = parser.parse_html(file_path)        
    if not parsed_html:
        return []
    tables = parser.extract_tables(parsed_html)            
    return tables


In [ ]:
# 파일 경로 (사용자 로컬 환경에 맞춰 경로를 조정해야 합니다)
file_path = '삼성전자_감사보고서_2014_2024/감사보고서_2014.htm'

if os.path.exists(file_path):
    # 함수를 호출하여 모든 테이블을 추출합니다.
    all_tables_2014 = browse_tables(file_path)

    print(f"파일: {file_path}")
    print(f"총 {len(all_tables_2014)}개의 테이블이 파싱되었습니다.")
    
    # 추출된 테이블들을 함수 외부에서 순차적으로 출력합니다.
    for i, df in enumerate(all_tables_2014):
        print(f"\n--- [테이블 {i+1}] ---")
        display(df)
else:
    print(f"파일을 찾을 수 없습니다: {file_path}")

In [ ]:

# 2014년부터 2024년까지 반복하여 11번째 테이블 추출 및 출력
for year in range(2014, 2025):
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    
    if os.path.exists(file_path):
        all_tables_for_year = browse_tables(file_path)
        
        # 11번째 테이블(인덱스 10)이 존재하는지 확인
        if len(all_tables_for_year) > 10:
            selected_table = all_tables_for_year[10]
            print(f"\n==================== {year}년 감사보고서의 11번째 테이블 ====================")
            display(selected_table)
        else:
            print(f"\n==================== {year}년 감사보고서 ====================")
            print(f"총 {len(all_tables_for_year)}개의 테이블이 파싱되었으며, 11번째 테이블이 없습니다.")
    else:
        print(f"\n파일을 찾을 수 없습니다: {file_path}")

In [ ]:

# 클래스 인스턴스 생성
parser = AuditReportParser()

# 2014년부터 2024년까지 반복
for year in range(2014, 2025):
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)
        
        if parsed_html:
            inventory_df = parser.extract_inventory_table(parsed_html)
            
            print(f"\n==================== {year}년 감사보고서 ====================")
            if inventory_df is not None:
                display(inventory_df)
                
                # 추출된 DataFrame을 CSV 파일로 저장
                csv_filename = f"outputs/재고자산_{year}.csv"
                inventory_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
                print(f"'{csv_filename}' 파일로 저장되었습니다.")
            else:
                print(f"재고자산 테이블을 찾을 수 없습니다.")
    else:
        print(f"\n파일을 찾을 수 없습니다: {file_path}")

In [ ]:
parser = AuditReportParser()

# 출력 파일을 저장할 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

for year in range(2014, 2025):
    # 파일 경로 설정 (경로를 사용자의 환경에 맞게 수정해주세요)
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    # 파일 존재 여부 확인 후 파싱 및 출력
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)
        
        if parsed_html:
            df = parser.extract_investment_changes(parsed_html)
            
            print(f"==================== {year}년 투자의 변동내역 파싱 결과 ====================")
            if df is not None:
                # DataFrame 출력
                display(df) 
                
                # DataFrame을 CSV 파일로 저장
                csv_filename = f"outputs/2-a {year}.csv"
                df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
                print(f"'{csv_filename}' 파일로 저장되었습니다.")
            else:
                print("테이블을 찾을 수 없습니다.")
    else:
        print(f"파일을 찾을 수 없습니다: {file_path}")

In [ ]:
parser = AuditReportParser()

# 출력 파일을 저장할 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

for year in range(2014, 2025):
    # 파일 경로 설정 (경로를 사용자의 환경에 맞게 수정해주세요)
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    # 파일 존재 여부 확인 후 파싱 및 출력
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)

    if parsed_html:
        df = parser.extract_major_investments(parsed_html)
        
        print(f"==================== {year}년 관계기업 투자 현황 파싱 결과 ====================")
        if df is not None:
            # DataFrame 출력
            display(df) 
            
            # DataFrame을 CSV 파일로 저장
            csv_filename = f"outputs/2-b {year}.csv"
            df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
            print(f"'{csv_filename}' 파일로 저장되었습니다.")
        else:
            print("테이블을 찾을 수 없습니다.")
else:
    print(f"파일을 찾을 수 없습니다: {file_path}")

In [ ]:
parser = AuditReportParser()

# 출력 파일을 저장할 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

for year in range(2014, 2025):
    # 파일 경로 설정 (경로를 사용자의 환경에 맞게 수정해주세요)
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    # 파일 존재 여부 확인 후 파싱 및 출력
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)

    if parsed_html:
        df = parser.extract_subsidiaries(parsed_html)
        
        print(f"==================== {year}년 종속기업 재무정보 파싱 결과 ====================")
        if df is not None:
            # DataFrame 출력
            display(df) 
            
            # DataFrame을 CSV 파일로 저장
            csv_filename = f"outputs/2-c-1 {year}.csv"
            df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
            print(f"'{csv_filename}' 파일로 저장되었습니다.")
        else:
            print("테이블을 찾을 수 없습니다.")
else:
    print(f"파일을 찾을 수 없습니다: {file_path}")

In [ ]:
# 클래스 인스턴스 생성
parser = AuditReportParser()

# 출력 파일을 저장할 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

# 2014년부터 2024년까지 반복
for year in range(2014, 2025):
    # 파일 경로 설정 (경로를 사용자의 환경에 맞게 수정해주세요)
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    
    # 파일명에서 연도 추출
    match = re.search(r'(\d{4})', os.path.basename(file_path))
    if match:
        current_year = int(match.group(1))
    else:
        current_year = None
    
    if os.path.exists(file_path) and current_year:
        parsed_html = parser.parse_html(file_path)
        
        if parsed_html:
            df = parser.extract_financial_info(parsed_html,current_year)
            
            print(f"==================== {current_year}년 관계기업 재무정보 파싱 결과 ====================")
            if df is not None:
                display(df) 
                csv_filename = f"outputs/2-d_{current_year}.csv"
                df.to_csv(csv_filename, index=True, encoding='utf-8-sig')
                print(f"'{csv_filename}' 파일로 저장되었습니다.")
            else:
                print("테이블을 찾을 수 없습니다.")
    else:
        print(f"\n파일을 찾을 수 없습니다: {file_path}")
        

In [ ]:
parser = AuditReportParser()

# 출력 파일을 저장할 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

for year in range(2014, 2025):
    # 파일 경로 설정 (경로를 사용자의 환경에 맞게 수정해주세요)
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    # 파일 존재 여부 확인 후 파싱 및 출력
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)

    if parsed_html:
        print(f"==================== {year}년 관계기업 투자주식의 내역 파싱 결과 ====================")
        df = parser.extract_specific_investment_table(parsed_html)
            
        if df is not None:
            display(df)
            csv_filename = f"outputs/2-e {year}.csv"
            df.to_csv(csv_filename, index=True, encoding='utf-8-sig')
            print(f"'{csv_filename}' 파일로 저장되었습니다.")
        else:
            print("테이블을 찾을 수 없습니다.")
else:
    print(f"파일을 찾을 수 없습니다: {file_path}")

In [ ]:
parser = AuditReportParser()

# 출력 파일을 저장할 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

for year in range(2014, 2025):
    # 파일 경로 설정 (경로를 사용자의 환경에 맞게 수정해주세요)
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    # 파일 존재 여부 확인 후 파싱 및 출력
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)

    if parsed_html:
        print(f"==================== {year}년 유형자산 변동내역 파싱 결과 ====================")
        df = parser.extract_tangible_assets_table(parsed_html)
            
        if df is not None:
            display(df)
            csv_filename = f"outputs/3-a {year}.csv"
            df.to_csv(csv_filename, index=True, encoding='utf-8-sig')
            print(f"'{csv_filename}' 파일로 저장되었습니다.")
        else:
            print("테이블을 찾을 수 없습니다.")
else:
    print(f"파일을 찾을 수 없습니다: {file_path}")

In [ ]:
parser = AuditReportParser()

# 출력 파일을 저장할 디렉토리 생성
if not os.path.exists('outputs'):
    os.makedirs('outputs')

for year in range(2014, 2025):
    # 파일 경로 설정 (경로를 사용자의 환경에 맞게 수정해주세요)
    file_path = f'삼성전자_감사보고서_2014_2024/감사보고서_{year}.htm'
    # 파일 존재 여부 확인 후 파싱 및 출력
    if os.path.exists(file_path):
        parsed_html = parser.parse_html(file_path)

    if parsed_html:
        print(f"==================== {year}년 무형자산 변동내역 파싱 결과 ====================")
        df = parser.extract_intangible_assets_table(parsed_html)
            
        if df is not None:
            display(df)
            csv_filename = f"outputs/3-b {year}.csv"
            df.to_csv(csv_filename, index=True, encoding='utf-8-sig')
            print(f"'{csv_filename}' 파일로 저장되었습니다.")
        else:
            print("테이블을 찾을 수 없습니다.")
else:
    print(f"파일을 찾을 수 없습니다: {file_path}")